# BIG TATA

In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col , sum as sum_func
from IPython.display import display


In [2]:
import findspark
findspark.init()
findspark.find()

'C:\\Program Files\\Spark\\spark-3.5.1-bin-hadoop3'

In [3]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("My Spark App") \
    .getOrCreate()

In [4]:
input_df = spark.read.csv("spotify_songs.csv", header=True, inferSchema=True)


In [5]:
# 3. Display schema of DataFrame. 
input_df.printSchema()


root
 |-- track_id: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- track_artist: string (nullable = true)
 |-- track_popularity: integer (nullable = true)
 |-- track_album_id: string (nullable = true)
 |-- track_album_name: string (nullable = true)
 |-- track_album_release_date: string (nullable = true)
 |-- playlist_name: string (nullable = true)
 |-- playlist_id: string (nullable = true)
 |-- playlist_genre: string (nullable = true)
 |-- playlist_subgenre: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: double (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- duration_ms: double (nullable = true)


In [6]:
# clean data

# remove unnecessary columns (track_album_id , track_album_name , track_album_release_date , playlist_name , playlist_id , track_popularity)

cleaned_df = input_df.drop("track_album_id","track_album_name","track_album_release_date","playlist_name","playlist_id" ,"track_popularity" , "track_name" , "track_artist")

# print schema
cleaned_df.printSchema()

# delete all records with null values




# show records count

print(cleaned_df.count())



root
 |-- track_id: string (nullable = true)
 |-- playlist_genre: string (nullable = true)
 |-- playlist_subgenre: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: double (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- duration_ms: double (nullable = true)
 |-- language: string (nullable = true)
 |-- lyrics: string (nullable = true)



18454


In [7]:


# #show records with null values

# columns = cleaned_df.columns

# for column in columns:
#     print ("records with null values in column " + column)
#     cleaned_df.filter(cleaned_df[column].isNull()).show()



In [13]:

# remove records with null values

cleaned_df = cleaned_df.na.drop()

# remove records with lyrics = NA
cleaned_df = cleaned_df.filter(cleaned_df.lyrics != "NA")
features = ["danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo","duration_ms"]

# make sure that all this features have numeric values

for feature in features:
    cleaned_df = cleaned_df.filter(col(feature).cast("float").isNotNull() | col(feature).cast("int").isNotNull() )


# get duration_ms first quartile

duaration_ms_data = cleaned_df.select("duration_ms").summary().toPandas().to_numpy()
print (duaration_ms_data)

first_quartile = float(duaration_ms_data[4][1])
third_quartile = float(duaration_ms_data[6][1])
duraton_IQR = (third_quartile - first_quartile) * 1.5

cleaned_df = cleaned_df.filter((col("duration_ms").cast("float") >= first_quartile - duraton_IQR) & (col("duration_ms").cast("float") <= third_quartile + duraton_IQR))


# show records count
print(cleaned_df.count())


[['count' '18184']
 ['mean' '230549.0757259129']
 ['stddev' '56735.896677169534']
 ['min' '31893.0']
 ['25%' '193594.0']
 ['50%' '221480.0']
 ['75%' '258027.0']
 ['max' '517810.0']]
17536


In [14]:



# show genre with more than 100 songs
result = cleaned_df.groupBy("playlist_genre").count()
filtered_result = result.filter(col("count") > 100).select("playlist_genre")
display(filtered_result.toPandas())


# remove all records that doesn't belong to filtered_result

cleaned_df = cleaned_df.join(filtered_result, on="playlist_genre", how="left")
print(cleaned_df.count())




,playlist_genre
0,r&b
1,pop
2,edm
3,rap
4,rock
5,latin


17536


In [15]:


# show subgenre with more than 100 songs
result = cleaned_df.groupBy("playlist_subgenre").count()
filtered_result = result.filter(col("count") > 100).select("playlist_subgenre")
display(filtered_result.toPandas())

# remove all records that doesn't belong to filtered_result

cleaned_df = cleaned_df.join(filtered_result, on="playlist_subgenre", how="left")

print(cleaned_df.count())


#export to csv
pandas_df = cleaned_df.toPandas()
pandas_df.to_csv("cleaned_songs.csv", index=False , header=True)








,playlist_subgenre
0,electropop
1,latin pop
2,tropical
3,album rock
4,new jack swing
5,urban contemporary
6,big room
7,pop edm
8,permanent wave
9,trap


17536
